In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D

In [ ]:
import json

# Open the JSON file
with open(r'C:\Users\nasci\OneDrive\Desktop\CS Projects\Python\PlatRecognitionChallenge\ML\plantnet_300K\plantnet300K_species_names.json') as f:
    data = json.load(f)

# Loop through the dictionary and print out each key-value pair
for key, value in data.items():
    print(key, value)

In [ ]:
train_dir = r"C:\Users\nasci\OneDrive\Desktop\CS Projects\Python\PlatRecognitionChallenge\ML\plantnet_300K\images_train"
test_dir = r'C:\Users\nasci\OneDrive\Desktop\CS Projects\Python\PlatRecognitionChallenge\ML\plantnet_300K\images_test'
val_dir = r"C:\Users\nasci\OneDrive\Desktop\CS Projects\Python\PlatRecognitionChallenge\ML\plantnet_300K\images_val"

In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [ ]:
# Create data generators for training, validation, and testing
train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical')

In [ ]:
strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")

In [ ]:
with strategy.scope():
    # Load the pre-trained InceptionV3 model
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))


    # Freeze the pre-trained layers
    for layer in base_model.layers:
        layer.trainable = False
        
    # Add your own layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(train_generator.num_classes, activation='softmax')(x)


    # Create the model
    model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)
    
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Train the model
    history = model.fit(
        train_generator,
        epochs=20,
        validation_data=val_generator)

In [ ]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)